Import neccessary libraries

In [515]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score,mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import train_test_split


Read data

In [516]:
train_data=pd.read_excel("JPN Data.xlsx")
train_data

,ID,CURR_AGE,GENDER,ANN_INCOME,AGE_CAR,PURCHASE
0,00001Q15YJ,50,M,445344.000000,439,0
1,00003I71CQ,35,M,107634.000000,283,0
2,00003N47FS,59,F,502786.666667,390,1
3,00005H41DE,43,M,585664.000000,475,0
4,00007E17UM,39,F,705722.666667,497,1
...,...,...,...,...,...,...
39995,99988B18OG,49,M,478511.000000,464,1
39996,99990Q44VP,28,M,271419.000000,61,0
39997,99997Q09VJ,41,F,590704.000000,379,1
39998,99998J59EE,50,M,363160.000000,302,1


In [517]:
test_data=pd.read_excel("IN_Data.xlsx")
test_data

,ID,CURR_AGE,GENDER,ANN_INCOME,DT_MAINT
0,20710B05XL,54,M,1425390,2018-04-20
1,89602T51HX,47,M,1678954,2018-06-08
2,70190Z52IP,60,M,931624,2017-07-31
3,25623V15MU,55,F,1106320,2017-07-31
4,36230I68CE,32,F,748465,2019-01-27
...,...,...,...,...,...
69995,35280V26PS,49,F,861770,2018-04-08
69996,79863N01VR,59,M,888976,2018-05-17
69997,94236O99QE,33,M,699676,2019-04-21
69998,23696V12DP,41,M,1881922,2018-07-14


Look for Null values and remove row corresponding to Null value

In [518]:
train_data.isnull().sum()

ID            0
CURR_AGE      0
GENDER        0
ANN_INCOME    0
AGE_CAR       0
PURCHASE      0
dtype: int64

In [519]:
test_data.isnull().sum()

ID            0
CURR_AGE      0
GENDER        0
ANN_INCOME    0
DT_MAINT      0
dtype: int64

No null value value found in both files

Create a new column in IND_Data.xlsx equivalent to have Car_age

In [520]:
def days(Date):
    given_date = datetime(2019, 7, 1)
    if pd.isna(Date):
        return None
    return (given_date - Date).days
test_data['DT_MAINT'] = pd.to_datetime(test_data['DT_MAINT'])
test_data['AGE_CAR'] = test_data['DT_MAINT'].apply(days)

Create a new column in JPN Data.xlsx and Ind_Data.xlsx to contain category details

In [521]:
def classify_age_car(age):
    if age < 200:
        return '1'
    elif 200 <= age <= 360:
        return '2'
    elif 360 < age <= 500:
        return '3'
    else:
        return '4'

train_data['AGE_CAR_CATEGORY'] = train_data['AGE_CAR'].apply(classify_age_car)
test_data['AGE_CAR_CATEGORY'] = test_data['AGE_CAR'].apply(classify_age_car)

Encode categorical variables

In [522]:
train_data.replace({'GENDER':{'M':0,'F':1},'AGE_CAR_CATEGORY':{'1':1,'2':2,'3':3,'4':4}},inplace=True)
test_data.replace({'GENDER':{'M':0,'F':1},'AGE_CAR_CATEGORY':{'1':1,'2':2,'3':3,'4':4}},inplace=True)

C:\Users\Kaustubha Ram\AppData\Local\Temp\ipykernel_20104\940576726.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data.replace({'GENDER':{'M':0,'F':1},'AGE_CAR_CATEGORY':{'1':1,'2':2,'3':3,'4':4}},inplace=True)
C:\Users\Kaustubha Ram\AppData\Local\Temp\ipykernel_20104\940576726.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data.replace({'GENDER':{'M':0,'F':1},'AGE_CAR_CATEGORY':{'1':1,'2':2,'3':3,'4':4}},inplace=True)


Create models

In [523]:
model1=LogisticRegression()
model2=RandomForestClassifier()
model3=CategoricalNB()


Drop uneccessary columns

In [524]:
train_data.drop(columns="ID",inplace=True)
test_data.drop(columns="ID",inplace=True)

Split train_data into x_train and y_train

In [525]:
train_data['ANN_INCOME']=pd.to_numeric(train_data['ANN_INCOME'])
x=train_data[['CURR_AGE', 'GENDER','ANN_INCOME', 'AGE_CAR_CATEGORY']]
y=train_data['PURCHASE']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1, random_state=20)

Fit the models

In [526]:
model1.fit(x_train,y_train)
model2.fit(x_train,y_train)
model3.fit(x_train,y_train)

c:\Users\Kaustubha Ram\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CategoricalNB()

Make predictions using x_test and y_test

In [527]:
log_pred=model1.predict(x_test)
ran_pred=model2.predict(x_test)
cat_pred=model3.predict(x_test)
x_test.shape

(4000, 4)

In [528]:
x_test_data = test_data[['CURR_AGE', 'GENDER','ANN_INCOME', 'AGE_CAR_CATEGORY']]
x_test_data.shape

(70000, 4)

Evaluate metrics for each model 

In [529]:
print("LogisticRegression: ")
print("accuracy_score: ",accuracy_score(log_pred,y_test))
print("r2_score: ",r2_score(log_pred,y_test))

print("RandomForestClassifier: ")
print("accuracy_score: ",accuracy_score(ran_pred,y_test))
print("r2_score: ",r2_score(ran_pred,y_test))

print("Categorical : ")
print("accuracy_score: ",accuracy_score(cat_pred,y_test))
print("r2_score: ",r2_score(cat_pred,y_test))

LogisticRegression: 
accuracy_score:  0.6705
r2_score:  -0.4741908937394146
RandomForestClassifier: 
accuracy_score:  0.65675
r2_score:  -0.4076983567966814
Categorical : 
accuracy_score:  0.68425
r2_score:  -0.277905489631056


In [530]:
train_data

,CURR_AGE,GENDER,ANN_INCOME,AGE_CAR,PURCHASE,AGE_CAR_CATEGORY
0,50,0,445344.000000,439,0,3
1,35,0,107634.000000,283,0,2
2,59,1,502786.666667,390,1,3
3,43,0,585664.000000,475,0,3
4,39,1,705722.666667,497,1,3
...,...,...,...,...,...,...
39995,49,0,478511.000000,464,1,3
39996,28,0,271419.000000,61,0,1
39997,41,1,590704.000000,379,1,3
39998,50,0,363160.000000,302,1,2


In [533]:
test_data['ANN_INCOME']=pd.to_numeric(test_data['ANN_INCOME'])
x_test_data = test_data[['CURR_AGE', 'GENDER','ANN_INCOME', 'AGE_CAR_CATEGORY']]
x_test_data.shape

(70000, 4)

Since Categorical Model gives highest accuracy of upto 68% it is used to predict the indian data

In [546]:
x_test = test_data[['CURR_AGE', 'GENDER', 'ANN_INCOME', 'AGE_CAR_CATEGORY']]
test_data['PREDICTED_PURCHASE_NB'] = model3.predict(x_test_data)


IndexError: index 1425390 is out of bounds for axis 1 with size 799971